# Installation & Setup for LangGraph with OpenSearch


In [1]:
## install langgraph-opensearch
!pip install langgraph-opensearch==0.1.5


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
os.environ["OSS_HOST"] = "184.72.125.181"
os.environ["OSS_PORT"] = "9200"

In [3]:
client_kwargs = {
    "hosts": [{"host": os.getenv('OSS_HOST'), "port": int(os.getenv('OSS_PORT'))}],
    "use_ssl": True,
    "http_auth": (os.environ.get("OSS_USER", "admin"), os.environ.get("OSS_PASS", "admin")),
    "verify_certs": False,
    "timeout": 30,
    "max_retries": 3,
}

In [11]:
import warnings
from urllib3.exceptions import InsecureRequestWarning
# Suppress the SSL warnings
warnings.filterwarnings("ignore", category=InsecureRequestWarning)

# Optionally suppress opensearch-py connection warnings
warnings.filterwarnings("ignore", category=UserWarning)


# Q&A with OpenSearch as Checkpoint Storage

In [12]:
from typing_extensions import TypedDict
from opensearchpy import OpenSearch
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.opensearch import OpenSearchSaver

from langchain_openai import ChatOpenAI

from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)

In [13]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.0,
    max_tokens=1000,
    streaming=True,
)

In [14]:
def ask(state: MessagesState) -> MessagesState:
    """
    Ask a question to the LLM and return the response.
    """
    question = state['messages'][-1].content
    response = llm.invoke(
        [*state['messages']]
    )
    return { 'messages': [ AIMessage(content=response.content) ] }

In [15]:
graph = StateGraph(MessagesState)
graph.add_node('ask', ask)
graph.add_edge(START, 'ask')
graph.add_edge('ask', END)

In [16]:
# Initialize OpenSearchSaver with the graph
client = OpenSearch(**client_kwargs)
checkpointer = OpenSearchSaver(client=client)
config = {
    'configurable': {
        'thread_id': '1'
    }
}
graph = graph.compile(checkpointer=checkpointer)

In [17]:
# Run the graph with an initial message
response = graph.invoke(
    {
        "messages": [
            HumanMessage(content="What is the capital of France?")
        ]
    },
    config
)
print(response['messages'][-1].content)

The capital of France is Paris.


In [18]:
response = graph.invoke(
    {
        "messages": [
            HumanMessage(content="What are its key attractions?")
        ]
    },
    config
)
print(response['messages'][-1].content)

Paris, the capital of France, is renowned for its rich history, culture, and architecture. Some of its key attractions include:

1. **Eiffel Tower**: An iconic symbol of Paris, offering stunning views of the city from its observation decks.

2. **Louvre Museum**: The world's largest art museum, home to thousands of works, including the Mona Lisa and the Venus de Milo.

3. **Notre-Dame Cathedral**: A masterpiece of French Gothic architecture, known for its stunning facade and beautiful stained glass windows.

4. **Champs-Élysées and Arc de Triomphe**: A famous avenue leading to the monumental arch, which honors those who fought and died for France.

5. **Sacré-Cœur Basilica**: Located at the highest point in the city, Montmartre, offering panoramic views of Paris.

6. **Palace of Versailles**: A short trip from Paris, this opulent palace is known for its magnificent gardens and the Hall of Mirrors.

7. **Seine River Cruises**: Offering a unique perspective of Paris's landmarks, includin

# Search Checkpoints

In [25]:
items = list(checkpointer.list(config))

In [26]:
items

[CheckpointTuple(config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f075106-201e-600b-8007-f0f046808d10'}}, checkpoint={'v': 4, 'ts': '2025-08-09T11:02:47.270196+00:00', 'id': '1f075106-201e-600b-8007-f0f046808d10', 'channel_values': {'messages': [HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}, id='1397faeb-9dbd-4ca8-9e31-f480313771cc'), AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={}, id='428e8ef8-58a2-4e5f-a3ad-075ae900c5ff'), HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}, id='323c1c87-e115-49bb-b713-4d0989b18c60'), AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={}, id='e74cce70-d032-430e-bff0-b252d410498f'), HumanMessage(content='What are its key attractions?', additional_kwargs={}, response_metadata={}, id='68cd1919-a2f3-4f7c-9a38-d7a6f9d92d42'), AIMessage(

# Replay from a Checkpoint

In [27]:
to_replay = items[0].config

In [31]:
for event in graph.stream(None, to_replay, stream_mode="values"):
    if "messages" in event:
        print(f"Messages:{event['messages']}")

Messages:[HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}, id='1397faeb-9dbd-4ca8-9e31-f480313771cc'), AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={}, id='428e8ef8-58a2-4e5f-a3ad-075ae900c5ff'), HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}, id='323c1c87-e115-49bb-b713-4d0989b18c60'), AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={}, id='e74cce70-d032-430e-bff0-b252d410498f'), HumanMessage(content='What are its key attractions?', additional_kwargs={}, response_metadata={}, id='68cd1919-a2f3-4f7c-9a38-d7a6f9d92d42'), AIMessage(content="Paris, the capital of France, is renowned for its rich history, culture, and architecture. Some of its key attractions include:\n\n1. **Eiffel Tower**: An iconic symbol of Paris, offering stunning views of the city from its observation decks.\n\n2. **Louvre Mus